In [1]:
# !pip install dart-fss
!pip -q install -U dart-fss pandas tqdm tenacity

In [28]:
# !pip install -U dart-fss pandas openpyxl tqdm

import os, re, time
from dataclasses import dataclass
from typing import Optional, Dict
from dotenv import load_dotenv
from datetime import datetime

import pandas as pd
from tqdm import tqdm
import dart_fss as dart

# =========================
# 0) Open DART API KEY 설정
# =========================
load_dotenv()
api_key = os.getenv("DART_API_KEY")
if api_key is not None:
    dart.set_api_key(api_key=api_key)
else:
    raise RuntimeError("DART_API_KEY 환경변수가 설정되지 않았습니다.")

SLEEP = 0.15  # API 호출 간격

# =========================
# 1) 대상 기업(표시명) 목록 - 1~10번
# =========================
TARGET_NAMES = [
    "삼성물산",      # 1번
    "효성중공업",    # 2번
    "현대건설",      # 3번
    "HJ중공업",      # 4번
    "DL이앤씨",      # 5번
    "GS건설",       # 6번
    "대우건설",      # 7번
    "HDC현대산업개발", # 8번
    "아이에스동서",   # 9번
    "태영건설"       # 10번
]

@dataclass
class CorpRef:
    corp_code: str
    corp_name: str
    stock_code: Optional[str] = None

# 고정 매핑으로 시작(초기값), 이후 앵커링 단계에서 교체/보정됨
corp_refs: Dict[str, CorpRef] = {
    "삼성물산": CorpRef("00126229", "삼성물산", "000830"),   # 이후 028260으로 교체 앵커링
    "효성중공업": CorpRef("01316245", "효성중공업", "298040"),
    "현대건설": CorpRef("00164478", "현대건설", "000720"),
    "HJ중공업": CorpRef("00633835", "HJ중공업", "097230"),
    "DL이앤씨": CorpRef("01524093", "DL이앤씨", "375500"),
    "GS건설": CorpRef("00120030", "GS건설", "006360"),
    "대우건설": CorpRef("00124540", "대우건설", "047040"),
    "HDC현대산업개발": CorpRef("01310269", "HDC현대산업개발", "294870"),
    "아이에스동서": CorpRef("00115977", "아이에스동서", "010780"),
    "태영건설": CorpRef("00535825", "태영건설", None),      # 이후 009410으로 교체 앵커링
}

# ==========================================
# 2) 보고서 코드 / 분기말 / 계정명 정규화 등
# ==========================================
REPRT = {"1Q":"11013","H1":"11012","3Q":"11014","ANNUAL":"11011"}
ORDER = ["1Q","H1","3Q","ANNUAL"]
Q_END = {"11013":"-03-31","11012":"-06-30","11014":"-09-30","11011":"-12-31"}

NOTE_TAIL = re.compile(r"\s*\([^)]*\)\s*\)*\s*$")
MULTISPACE = re.compile(r"\s+")
def clean_account_name(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.strip()
    s = s.replace("수익 (매출액)", "매출액")
    s = NOTE_TAIL.sub("", s)
    s = MULTISPACE.sub(" ", s)
    s = s.replace("영업이익(손실)", "영업이익")
    s = s.replace("분기(중간)순이익", "분기순이익")
    s = s.replace("당기순이익(손실)", "당기순이익")
    return s

TARGET_ACCOUNTS = {
    "자산총계": ["자산총계", "총자산"],
    "부채총계": ["부채총계", "총부채"],
    "자본총계": [
        "자본총계", "총자본", "자본총계(지배+비지배)"
    ],
    # 손익(가능하면 보수적으로 우선순위)
    "매출액": ["매출액", "매출", "매출수익", "영업수익", "수익"],
    "영업이익": ["영업이익", "영업손익"],
    "당기순이익": ["분기순이익", "당기순이익", "반기순이익"],
}

def parse_amount(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if s in ("", "-", "None", "nan"):
        return None
    s = s.replace(",", "")
    if s.startswith("(") and s.endswith(")"):  # (1,234) → -1234
        s = "-" + s[1:-1]
    try:
        return float(s)
    except:
        return None

# =========================================================
# 3) 주식코드로 정확한 corp_code를 찾는 앵커링 함수
# =========================================================
def _resolve_by_stock(stock_code: str):
    """
    주식코드로 corp_list를 순회해서 정확한 corp_code/현재공식명칭을 찾음
    """
    cl = dart.get_corp_list()
    for c in cl:
        if getattr(c, "stock_code", None) == stock_code:
            return str(getattr(c, "corp_code")).zfill(8), getattr(c, "corp_name", "")
    raise ValueError(f"stock_code '{stock_code}' 회사를 찾지 못했습니다.")

# ---------------------------------------------------------
# 메인 표시명 중 삼성물산/태영건설을 올바른 엔트리로 교체 앵커링
#   - 삼성물산: 028260 (합병 후 신주)
#   - 태영건설: 009410
# ---------------------------------------------------------
PRIMARY_STOCK_ANCHOR = {
    "삼성물산": "028260",
    "태영건설": "009410",
}
for nm, sc in PRIMARY_STOCK_ANCHOR.items():
    try:
        cc, now_official = _resolve_by_stock(sc)
        corp_refs[nm] = CorpRef(cc, nm, sc)  # 표시명은 그대로 유지
        print(f"[anchor primary] {nm} -> corp_code={cc}, stock_code={sc}, now_official='{now_official}'")
    except Exception as e:
        print(f"[warn] anchor primary {nm}: {e}")

# ---------------------------------------------------------
# 레거시 명칭(라인리지)도 주식코드로 고정
#   - 대림산업: 000210
#   - 현대산업개발: 012630
# ---------------------------------------------------------
LEGACY_STOCK_ANCHOR = {
    "대림산업": "000210",
    "현대산업개발": "012630",
}
for legacy_name, sc in LEGACY_STOCK_ANCHOR.items():
    try:
        cc, now_official = _resolve_by_stock(sc)
        corp_refs[legacy_name] = CorpRef(cc, legacy_name, sc)
        print(f"[anchor legacy] {legacy_name} -> corp_code={cc}, stock_code={sc}, now_official='{now_official}'")
    except Exception as e:
        print(f"[warn] anchor legacy {legacy_name}: {e}")

# ==========================================
# 4) FS 고정 + sj_div 필터 + 계정 매칭 강화 버전
# ==========================================
def fetch_report_accounts_strict(corp_code: str, year: int, reprt_code: str, fs_div: str):
    """
    단일 보고서 로드 (fs_div 고정: 'CFS' 또는 'OFS')
    - 손익 항목은 IS/CIS에서만, 재무상태 항목은 BS에서만 매칭
    - 반환: 타겟 6계정 + report_date
    """
    def _call(cc, yy, rc, div):
        time.sleep(SLEEP)
        return dart.api.finance.fnltt_singl_acnt_all(
            corp_code=str(cc).zfill(8), bsns_year=str(yy), reprt_code=rc, fs_div=div
        )

    resp = _call(corp_code, year, reprt_code, fs_div)
    items = resp.get("list", []) if isinstance(resp, dict) else (resp or [])
    df = pd.DataFrame(items)
    if df.empty:
        return {k: None for k in TARGET_ACCOUNTS} | {"report_date": f"{year}{Q_END[reprt_code]}"}

    # 정규화
    df["account_nm_clean"] = df["account_nm"].astype(str).map(clean_account_name)
    if "sj_div" not in df.columns:
        df["sj_div"] = None
    amt_col = "thstrm_amount" if "thstrm_amount" in df.columns else None

    # 보고일
    if "thstrm_dt" in df.columns and pd.notna(df["thstrm_dt"]).any():
        rpt_dt = str(df["thstrm_dt"].dropna().iloc[0])
    else:
        rpt_dt = f"{year}{Q_END[reprt_code]}"

    # 필터 함수: IS/CIS/BS (add_amount 우선, 없으면 누적금액 사용)
    def pick_amount(cands, allowed_sj=("IS","CIS","BS")):
        sub = df[df["account_nm_clean"].isin(cands)]
        if allowed_sj is not None:
            sub = sub[sub["sj_div"].isin(allowed_sj)]
        if sub.empty:
            return None
        
        row = sub.sort_values("ord").iloc[0] if "ord" in sub.columns else sub.iloc[0]
        
        # add_amount(분기금액)가 있으면 우선 사용
        if "thstrm_add_amount" in df.columns and pd.notna(row.get("thstrm_add_amount")):
            add_val = parse_amount(row["thstrm_add_amount"])
            if add_val is not None:
                return add_val
        
        # add_amount가 없거나 null이면 누적금액(amount) 사용
        if amt_col and pd.notna(row.get(amt_col)):
            return parse_amount(row[amt_col])
        
        return None

    out = {}
    # 재무상태표: BS만
    out["자산총계"] = pick_amount(TARGET_ACCOUNTS["자산총계"], allowed_sj=("BS",))
    out["부채총계"] = pick_amount(TARGET_ACCOUNTS["부채총계"], allowed_sj=("BS",))
    out["자본총계"] = pick_amount(TARGET_ACCOUNTS["자본총계"], allowed_sj=("BS",))
    # 손익계정: IS/CIS만
    out["매출액"]   = pick_amount(TARGET_ACCOUNTS["매출액"],   allowed_sj=("IS","CIS"))
    out["영업이익"] = pick_amount(TARGET_ACCOUNTS["영업이익"], allowed_sj=("IS","CIS"))
    out["당기순이익"] = pick_amount(TARGET_ACCOUNTS["당기순이익"], allowed_sj=("IS","CIS"))

    out["report_date"] = rpt_dt
    return out

def _probe_fs_div_for_year(corp_code: str, year: int) -> str:
    """
    연도 내 대표 보고서(우선 ANNUAL, 없으면 3Q/H1/1Q 순)에서 CFS 가용 여부 확인.
    CFS 가 하나라도 있으면 CFS 고정, 아니면 OFS.
    """
    probe_order = [REPRT["ANNUAL"], REPRT["3Q"], REPRT["H1"], REPRT["1Q"]]
    for rc in probe_order:
        try:
            time.sleep(SLEEP)
            r = dart.api.finance.fnltt_singl_acnt_all(
                corp_code=str(corp_code).zfill(8), bsns_year=str(year), reprt_code=rc, fs_div="CFS"
            )
            lst = r.get("list", []) if isinstance(r, dict) else (r or [])
            if lst:
                return "CFS"
        except Exception:
            pass
    return "OFS"

def yearly_quarter_table_with_fs_lock(corp_name: str, corp_code: str, year: int) -> pd.DataFrame:
    """
    (중요) 연도별 FS 기준을 CFS/OFS 중 하나로 '고정'하여 1Q/H1/3Q/ANNUAL 전체를 조회.
    손익 항목: add_amount(분기금액) 우선, 없으면 누적→분기 변환(차분) 수행.
    """
    fs_fixed = _probe_fs_div_for_year(corp_code, year)

    raw, dates = {}, {}
    for tag in ORDER:
        try:
            rec = fetch_report_accounts_strict(corp_code, year, REPRT[tag], fs_div=fs_fixed)
        except Exception:
            rec = {k: None for k in TARGET_ACCOUNTS} | {"report_date": f"{year}{Q_END[REPRT[tag]]}"}
        raw[tag] = rec
        dates[tag] = rec.get("report_date")

    def d(a, b):
        return None if (a is None or b is None) else a - b

    # 손익 항목: add_amount 우선 처리는 fetch_report_accounts_strict에서 완료됨
    # 여기서는 기본 누적→분기 차분 방식 적용 (add_amount가 없는 경우 대비)
    sales_q = {
        "Q1": raw["1Q"]["매출액"],
        "Q2": d(raw["H1"]["매출액"], raw["1Q"]["매출액"]),
        "Q3": d(raw["3Q"]["매출액"], raw["H1"]["매출액"]),
        "Q4": d(raw["ANNUAL"]["매출액"], raw["3Q"]["매출액"]),
    }
    op_q = {
        "Q1": raw["1Q"]["영업이익"],
        "Q2": d(raw["H1"]["영업이익"], raw["1Q"]["영업이익"]),
        "Q3": d(raw["3Q"]["영업이익"], raw["H1"]["영업이익"]),
        "Q4": d(raw["ANNUAL"]["영업이익"], raw["3Q"]["영업이익"]),
    }
    ni_q = {
        "Q1": raw["1Q"]["당기순이익"],
        "Q2": d(raw["H1"]["당기순이익"], raw["1Q"]["당기순이익"]),
        "Q3": d(raw["3Q"]["당기순이익"], raw["H1"]["당기순이익"]),
        "Q4": d(raw["ANNUAL"]["당기순이익"], raw["3Q"]["당기순이익"]),
    }

    # 재무상태표(분기말 잔액 그대로)
    assets_q = {"Q1": raw["1Q"]["자산총계"], "Q2": raw["H1"]["자산총계"], "Q3": raw["3Q"]["자산총계"], "Q4": raw["ANNUAL"]["자산총계"]}
    liab_q   = {"Q1": raw["1Q"]["부채총계"], "Q2": raw["H1"]["부채총계"], "Q3": raw["3Q"]["부채총계"], "Q4": raw["ANNUAL"]["부채총계"]}
    equity_q = {"Q1": raw["1Q"]["자본총계"], "Q2": raw["H1"]["자본총계"], "Q3": raw["3Q"]["자본총계"], "Q4": raw["ANNUAL"]["자본총계"]}
    date_q   = {"Q1": dates["1Q"], "Q2": dates["H1"], "Q3": dates["3Q"], "Q4": dates["ANNUAL"]}

    rows = []
    for q in ["Q1","Q2","Q3","Q4"]:
        rows.append({
            "corp_name": corp_name,
            "corp_code": str(corp_code).zfill(8),
            "year": year,
            "quarter": q,
            "report_date": date_q[q],
            "자산총계": assets_q[q],
            "부채총계": liab_q[q],
            "자본총계": equity_q[q],
            "매출액": sales_q[q],
            "영업이익": op_q[q],
            "분기순이익": ni_q[q],
        })
    return pd.DataFrame(rows)

# ==========================================
# 5) 라인리지 폴백 (DL/HDC만 적용) + FS고정 함수 사용
# ==========================================
LINEAGE_RULES = {
    "DL이앤씨": ("대림산업", 2020),         # 2020년까지 레거시 '대림산업' 사용
    "HDC현대산업개발": ("현대산업개발", 2017), # 2017년까지 레거시 '현대산업개발' 사용
}

def yearly_quarter_table_with_lineage(pretty_name: str, corp_code: str, year: int) -> pd.DataFrame:
    base = yearly_quarter_table_with_fs_lock(pretty_name, corp_code, year)  #  FS고정 버전 사용
    num_cols = ["자산총계","부채총계","자본총계","매출액","영업이익","분기순이익"]
    all_missing = base[num_cols].isna().all().all()

    # 레거시 폴백: 해당 연도 전체가 결측이고 룰에 걸리면 레거시 주식코드 앵커로 다시 조회
    if all_missing:
        rule = LINEAGE_RULES.get(pretty_name)
        if rule and year <= rule[1]:
            legacy_name = rule[0]
            if legacy_name not in corp_refs:
                raise RuntimeError(f"legacy '{legacy_name}' corp_refs 미정의")
            alt = yearly_quarter_table_with_fs_lock(legacy_name, corp_refs[legacy_name].corp_code, year)
            alt["corp_name"] = pretty_name
            alt["corp_code"] = str(corp_code).zfill(8)
            return alt

    return base

# ==========================================
# 6) 날짜 범위 설정 (2015년 4분기 ~ 2025년 2분기)
# ==========================================
START_YEAR_LOAD = 2014  # 2014년부터 로드하여 2015년부터 사용
CURRENT_YEAR = datetime.now().year
CURRENT_MONTH = datetime.now().month

# 현재 월에 따른 종료 연도/분기 결정
# 분기 매핑: 1-3월=1분기, 4-6월=2분기, 7-9월=3분기, 10-12월=4분기
if CURRENT_MONTH <= 3:  # 1-3월이면 전년도 4분기까지
    END_YEAR = CURRENT_YEAR - 1
    MAX_QUARTER_CURRENT_YEAR = None
elif CURRENT_MONTH <= 6:  # 4-6월이면 올해 1분기까지
    END_YEAR = CURRENT_YEAR
    MAX_QUARTER_CURRENT_YEAR = "Q1"
elif CURRENT_MONTH <= 9:  # 7-9월이면 올해 2분기까지
    END_YEAR = CURRENT_YEAR
    MAX_QUARTER_CURRENT_YEAR = "Q2"
else:  # 10-12월이면 올해 3분기까지
    END_YEAR = CURRENT_YEAR
    MAX_QUARTER_CURRENT_YEAR = "Q3"

print(f"데이터 수집 범위: 2015년 Q4 ~ {END_YEAR}년 {MAX_QUARTER_CURRENT_YEAR if MAX_QUARTER_CURRENT_YEAR else 'Q4'}")

NUM_COLS = ["자산총계","부채총계","자본총계","매출액","영업이익","분기순이익"]

# ==========================================
# 7) 1~10번 기업 데이터 수집
# ==========================================
print(f"\n=== 1~10번 기업 데이터 수집 시작 (2015년 Q4 ~ {END_YEAR}년) ===")

all_dfs = []
for nm in tqdm(TARGET_NAMES, desc="1-10번 Companies"):
    ref = corp_refs[nm]
    for yy in range(START_YEAR_LOAD, END_YEAR+1):
        try:
            df_y = yearly_quarter_table_with_lineage(nm, ref.corp_code, yy)
            all_dfs.append(df_y)
        except Exception as e:
            print(f"[warn] {nm} {yy}: {e}")

result_df_1_10 = pd.concat(all_dfs, ignore_index=True)
result_df_1_10 = result_df_1_10[result_df_1_10["year"] >= 2015].copy()  # 2015년부터 사용

COL_ORDER = ["corp_name","corp_code","year","quarter","report_date"] + NUM_COLS
result_df_1_10 = result_df_1_10[COL_ORDER].sort_values(["corp_name","year","quarter"]).reset_index(drop=True)

# 2015년 1~3분기 제거 (2015년 4분기부터 시작)
mask_2015_q1_q3 = (result_df_1_10["year"] == 2015) & (result_df_1_10["quarter"].isin(["Q1", "Q2", "Q3"]))

# 현재년도 제한 적용
if MAX_QUARTER_CURRENT_YEAR and END_YEAR == CURRENT_YEAR:
    if MAX_QUARTER_CURRENT_YEAR == "Q2":
        mask_current_year_limit = (result_df_1_10["year"] == CURRENT_YEAR) & (result_df_1_10["quarter"].isin(["Q3", "Q4"]))
    elif MAX_QUARTER_CURRENT_YEAR == "Q3":
        mask_current_year_limit = (result_df_1_10["year"] == CURRENT_YEAR) & (result_df_1_10["quarter"].isin(["Q4"]))
    else:
        mask_current_year_limit = pd.Series([False] * len(result_df_1_10))
    
    to_drop_mask = mask_2015_q1_q3 | mask_current_year_limit
else:
    to_drop_mask = mask_2015_q1_q3

# 제거 대상 행 확인 및 제거
to_drop = result_df_1_10.loc[to_drop_mask, ["corp_name", "year", "quarter"]].copy()
if len(to_drop) > 0:
    print(f"제거 대상: {len(to_drop)}개 레코드")

result_df_1_10 = result_df_1_10.loc[~to_drop_mask].reset_index(drop=True)
result_df_1_10 = result_df_1_10.sort_values(["corp_name", "year", "quarter"]).reset_index(drop=True)

# ==========================================
# 8) 1~10번 기업 저장 (CSV / XLSX)
# ==========================================
os.makedirs("./dart_out", exist_ok=True)
csv_path_1_10  = "./dart_out/건설업1~10번_2015~2025_연결_분기재무_정규화.csv"
xlsx_path_1_10 = "./dart_out/건설업1~10번_2015~2025_연결_분기재무_정규화.xlsx"

result_df_1_10.to_csv(csv_path_1_10, index=False, encoding="utf-8-sig")
with pd.ExcelWriter(xlsx_path_1_10) as w:
    result_df_1_10.to_excel(w, sheet_name="1-10번_건설업_분기재무", index=False)

print(f"1~10번 기업 데이터 저장 완료: {result_df_1_10.shape}")
print(f"CSV: {csv_path_1_10}")
print(f"XLSX: {xlsx_path_1_10}")

# ==========================================
# 9) 결측 요약 및 샘플 (1~10번)
# ==========================================
missing_1_10 = (
    result_df_1_10
    .assign(_miss=result_df_1_10[NUM_COLS].isna().all(axis=1))
    .query("_miss == True")[["corp_name","year","quarter"]]
    .groupby(["corp_name","year"]).agg(missing_quarters=("quarter","unique")).reset_index()
    .sort_values(["corp_name","year"])
)

print(f"\n== 1~10번 기업 결측 요약 ==")
if len(missing_1_10) > 0:
    print(missing_1_10.head(10).to_string(index=False))
else:
    print("결측 데이터 없음")

# 1~10번 기업별 분포
print(f"\n1~10번 기업별 레코드 분포:")
for corp_name, count in result_df_1_10['corp_name'].value_counts().items():
    corp_num = TARGET_NAMES.index(corp_name) + 1
    print(f"  {corp_num}. {corp_name}: {count}개")

print("\n 1~10번 기업 데이터 수집 및 저장 완료!")

[anchor primary] 삼성물산 -> corp_code=00149655, stock_code=028260, now_official='삼성물산'
[anchor primary] 태영건설 -> corp_code=00153861, stock_code=009410, now_official='태영건설'
[anchor legacy] 대림산업 -> corp_code=00109693, stock_code=000210, now_official='DL'
[anchor legacy] 현대산업개발 -> corp_code=00164636, stock_code=012630, now_official='HDC'
데이터 수집 범위: 2015년 Q4 ~ 2025년 Q2

=== 1~10번 기업 데이터 수집 시작 (2015년 Q4 ~ 2025년) ===


1-10번 Companies: 100%|██████████| 10/10 [04:59<00:00, 30.00s/it]



제거 대상: 50개 레코드
1~10번 기업 데이터 저장 완료: (390, 11)
CSV: ./dart_out/건설업1~10번_2015~2025_연결_분기재무_정규화.csv
XLSX: ./dart_out/건설업1~10번_2015~2025_연결_분기재무_정규화.xlsx

== 1~10번 기업 결측 요약 ==
corp_name  year missing_quarters
HDC현대산업개발  2015             [Q4]
HDC현대산업개발  2016 [Q1, Q2, Q3, Q4]
HDC현대산업개발  2017         [Q1, Q2]
HDC현대산업개발  2018             [Q1]
    효성중공업  2015             [Q4]
    효성중공업  2016 [Q1, Q2, Q3, Q4]
    효성중공업  2017 [Q1, Q2, Q3, Q4]
    효성중공업  2018             [Q1]

1~10번 기업별 레코드 분포:
  5. DL이앤씨: 39개
  6. GS건설: 39개
  8. HDC현대산업개발: 39개
  4. HJ중공업: 39개
  7. 대우건설: 39개
  1. 삼성물산: 39개
  9. 아이에스동서: 39개
  10. 태영건설: 39개
  3. 현대건설: 39개
  2. 효성중공업: 39개

 1~10번 기업 데이터 수집 및 저장 완료!


C:\Users\baesh\AppData\Local\Temp\ipykernel_17552\1628854713.py:375: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df_1_10 = pd.concat(all_dfs, ignore_index=True)


In [25]:
# =========================
# 11~21번 기업들 (기존 추가 11개 기업)
# =========================

# 11~21번 기업들을 위한 새로운 리스트
COMPANIES_11_21 = [
    "서희건설",      # 11번
    "동원개발",      # 12번
    "코오롱글로벌",   # 13번
    "계룡건설",      # 14번
    "티케이케미칼",   # 15번
    "금호건설",      # 16번
    "이수화학",      # 17번
    "경동인베스트",   # 18번
    "자이에스앤디",   # 19번
    "동부건설",      # 20번
    "진흥기업"       # 21번
]

# 11~21번 기업들의 주식코드 앵커링 (수정된 코드 반영)
STOCK_ANCHOR_11_21 = {
    "서희건설": "035890",     # 11번 (수정: 019440 → 035890)
    "동원개발": "013120",     # 12번
    "코오롱글로벌": "003070",  # 13번
    "계룡건설": "013580",     # 14번
    "티케이케미칼": "104480",  # 15번
    "금호건설": "002990",     # 16번 (수정: 금고건설 → 금호건설, 014620 → 002990)
    "이수화학": "005950",     # 17번
    "경동인베스트": "012320",  # 18번 (수정: 092040 → 012320)
    "자이에스앤디": "317400",  # 19번 (수정: 319400 → 317400)
    "동부건설": "005960",     # 20번
    "진흥기업": "002780"      # 21번
}

# 11~21번 기업들 설정 확인 및 corp_refs 생성
if 'CorpRef' in globals():
    print(" CorpRef 클래스가 정의되어 있습니다. 11~21번 기업 설정을 진행합니다.")
    
    # corp_refs 생성 (11~21번)
    corp_refs_11_21 = {name: CorpRef(None, name, None) for name in COMPANIES_11_21}
    
    # 11~21번 기업들 앵커링 수행
    print("=== 11~21번 기업들 앵커링 수행 ===")
    if '_resolve_by_stock' in globals():
        for nm, sc in STOCK_ANCHOR_11_21.items():
            try:
                cc, now_official = _resolve_by_stock(sc)
                corp_refs_11_21[nm] = CorpRef(cc, nm, sc)
                print(f"[anchor 11-21] {nm} -> corp_code={cc}, stock_code={sc}, now_official='{now_official}'")
            except Exception as e:
                print(f"[warn] anchor 11-21 {nm}: {e}")

        print(f"\n11~21번 대상 기업 수: {len(COMPANIES_11_21)}개")
    else:
        print(" _resolve_by_stock 함수가 정의되지 않았습니다.")
else:
    print(" CorpRef 클래스가 정의되지 않았습니다. 먼저 기본 셀을 실행해주세요.")

# =========================
# 11~21번 기업들 데이터 수집 시작
# =========================
print(f"\n=== 11~21번 기업 데이터 수집 시작 (2015년 Q4 ~ {END_YEAR}년) ===")

companies_11_21_dfs = []

for nm in tqdm(COMPANIES_11_21, desc="11-21번 Companies"):
    ref = corp_refs_11_21[nm]
    if ref.corp_code is None:
        print(f"{nm} - corp_code가 없음")
        continue
        
    print(f"\n{nm} 데이터 수집 중...")
    
    for yy in range(START_YEAR_LOAD, END_YEAR+1):
        try:
            df_y = yearly_quarter_table_with_lineage(nm, ref.corp_code, yy)
            companies_11_21_dfs.append(df_y)
        except Exception as e:
            print(f" {nm} {yy}년 실패: {e}")

# 11~21번 기업들 데이터 결합
if companies_11_21_dfs:
    result_df_11_21 = pd.concat(companies_11_21_dfs, ignore_index=True)
    result_df_11_21 = result_df_11_21[result_df_11_21["year"] >= 2015].copy()  # 2015년부터 사용
    
    # 컬럼 순서 맞추기
    result_df_11_21 = result_df_11_21[COL_ORDER].sort_values(["corp_name","year","quarter"]).reset_index(drop=True)
    
    # 2015년 1~3분기 제거 (2015년 4분기부터 시작)
    mask_2015_q1_q3_11_21 = (result_df_11_21["year"] == 2015) & (result_df_11_21["quarter"].isin(["Q1", "Q2", "Q3"]))
    
    # 현재년도 제한 적용
    if MAX_QUARTER_CURRENT_YEAR and END_YEAR == CURRENT_YEAR:
        if MAX_QUARTER_CURRENT_YEAR == "Q2":
            mask_current_year_limit_11_21 = (result_df_11_21["year"] == CURRENT_YEAR) & (result_df_11_21["quarter"].isin(["Q3", "Q4"]))
        elif MAX_QUARTER_CURRENT_YEAR == "Q3":
            mask_current_year_limit_11_21 = (result_df_11_21["year"] == CURRENT_YEAR) & (result_df_11_21["quarter"].isin(["Q4"]))
        else:
            mask_current_year_limit_11_21 = pd.Series([False] * len(result_df_11_21))
        
        to_drop_mask_11_21 = mask_2015_q1_q3_11_21 | mask_current_year_limit_11_21
    else:
        to_drop_mask_11_21 = mask_2015_q1_q3_11_21
    
    # 제거 대상 행 확인 및 제거
    to_drop_11_21 = result_df_11_21.loc[to_drop_mask_11_21, ["corp_name", "year", "quarter"]].copy()
    if len(to_drop_11_21) > 0:
        print(f" 11~21번 제거 대상: {len(to_drop_11_21)}개 레코드")
    
    result_df_11_21 = result_df_11_21.loc[~to_drop_mask_11_21].reset_index(drop=True)
    result_df_11_21 = result_df_11_21.sort_values(["corp_name", "year", "quarter"]).reset_index(drop=True)
    
    print(f"\n 11~21번 기업들 수집 완료!")
    print(f"   총 레코드: {len(result_df_11_21)}개")
    print(f"   기업 수: {result_df_11_21['corp_name'].nunique()}개")
    print(f"   연도 범위: {result_df_11_21['year'].min()}~{result_df_11_21['year'].max()}")
else:
    print(" 11~21번 기업들 데이터 수집 실패")

# 11~21번 기업들 데이터 저장
if 'result_df_11_21' in locals() and not result_df_11_21.empty:
    # 결측치 확인
    financial_cols = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']
    missing_11_21 = result_df_11_21[result_df_11_21[financial_cols].isna().any(axis=1)]
    
    if len(missing_11_21) > 0:
        print(f"\n 11~21번 결측치 현황: {len(missing_11_21)}개 레코드")
    
    # CSV 및 엑셀 파일 저장
    csv_path_11_21 = "./dart_out/건설업11~21번_2015~2025_연결_분기재무_정규화.csv"
    xlsx_path_11_21 = "./dart_out/건설업11~21번_2015~2025_연결_분기재무_정규화.xlsx"
    
    # CSV 저장
    result_df_11_21.to_csv(csv_path_11_21, index=False, encoding='utf-8-sig')
    print(f"\n 11~21번 CSV 저장: {csv_path_11_21}")
    
    # 엑셀 저장
    with pd.ExcelWriter(xlsx_path_11_21, engine='openpyxl') as w:
        result_df_11_21.to_excel(w, sheet_name='11-21번_건설업_분기재무', index=False)
    print(f" 11~21번 엑셀 저장: {xlsx_path_11_21}")
    
    print(f"\n 11~21번 기업들 최종 데이터:")
    print(f"   정리된 레코드: {len(result_df_11_21)}개")
    print(f"   기업별 분포:")
    for corp_name, count in result_df_11_21['corp_name'].value_counts().items():
        corp_num = COMPANIES_11_21.index(corp_name) + 11
        print(f"     {corp_num}. {corp_name}: {count}개")
        

 CorpRef 클래스가 정의되어 있습니다. 11~21번 기업 설정을 진행합니다.
=== 11~21번 기업들 앵커링 수행 ===
[anchor 11-21] 서희건설 -> corp_code=00219848, stock_code=035890, now_official='서희건설'
[anchor 11-21] 동원개발 -> corp_code=00117966, stock_code=013120, now_official='동원개발'
[anchor 11-21] 코오롱글로벌 -> corp_code=00152880, stock_code=003070, now_official='코오롱글로벌'
[anchor 11-21] 계룡건설 -> corp_code=00102432, stock_code=013580, now_official='계룡건설산업'
[anchor 11-21] 티케이케미칼 -> corp_code=00693554, stock_code=104480, now_official='티케이케미칼'
[anchor 11-21] 금호건설 -> corp_code=00106313, stock_code=002990, now_official='금호건설'
[anchor 11-21] 이수화학 -> corp_code=00145552, stock_code=005950, now_official='이수화학'
[anchor 11-21] 경동인베스트 -> corp_code=00143527, stock_code=012320, now_official='경동인베스트'
[anchor 11-21] 자이에스앤디 -> corp_code=00367844, stock_code=317400, now_official='자이에스앤디'
[anchor 11-21] 동부건설 -> corp_code=00115612, stock_code=005960, now_official='동부건설'
[anchor 11-21] 진흥기업 -> corp_code=00150828, stock_code=002780, now_official='진흥기업'

11~21번 

11-21번 Companies:   0%|          | 0/11 [00:00<?, ?it/s]


서희건설 데이터 수집 중...


11-21번 Companies:   9%|▉         | 1/11 [00:26<04:29, 26.99s/it]


동원개발 데이터 수집 중...


...


동부건설 데이터 수집 중...


11-21번 Companies:  91%|█████████ | 10/11 [04:47<00:28, 28.56s/it]


진흥기업 데이터 수집 중...


11-21번 Companies: 100%|██████████| 11/11 [05:22<00:00, 29.35s/it]
C:\Users\baesh\AppData\Local\Temp\ipykernel_17552\163029915.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df_11_21 = pd.concat(companies_11_21_dfs, ignore_index=True)

C:\Users\baesh\AppData\Local\Temp\ipykernel_17552\163029915.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df_11_21 = pd.concat(companies_11_21_dfs, ignore_index=True)


 11~21번 제거 대상: 55개 레코드

 11~21번 기업들 수집 완료!
   총 레코드: 429개
   기업 수: 11개
   연도 범위: 2015~2025

 11~21번 결측치 현황: 81개 레코드

 11~21번 CSV 저장: ./dart_out/건설업11~21번_2015~2025_연결_분기재무_정규화.csv
 11~21번 엑셀 저장: ./dart_out/건설업11~21번_2015~2025_연결_분기재무_정규화.xlsx

 11~21번 기업들 최종 데이터:
   정리된 레코드: 429개
   기업별 분포:
     18. 경동인베스트: 39개
     14. 계룡건설: 39개
     16. 금호건설: 39개
     20. 동부건설: 39개
     12. 동원개발: 39개
     11. 서희건설: 39개
     17. 이수화학: 39개
     19. 자이에스앤디: 39개
     21. 진흥기업: 39개
     13. 코오롱글로벌: 39개
     15. 티케이케미칼: 39개
 11~21번 엑셀 저장: ./dart_out/건설업11~21번_2015~2025_연결_분기재무_정규화.xlsx

 11~21번 기업들 최종 데이터:
   정리된 레코드: 429개
   기업별 분포:
     18. 경동인베스트: 39개
     14. 계룡건설: 39개
     16. 금호건설: 39개
     20. 동부건설: 39개
     12. 동원개발: 39개
     11. 서희건설: 39개
     17. 이수화학: 39개
     19. 자이에스앤디: 39개
     21. 진흥기업: 39개
     13. 코오롱글로벌: 39개
     15. 티케이케미칼: 39개


In [26]:
# =========================
# 22~32번 기업들 
# =========================

# 22~32번 기업들을 위한 새로운 리스트
COMPANIES_22_32 = [
    "KCC건설",      # 22번 
    "HS화성",      # 23번
    "서한",        # 24번
    "HL D&I",      # 25번
    "한신공영",     # 26번
    "남광토건",     # 27번
    "삼부토건",     # 28번
    "우원개발",     # 29번
    "대원",        # 30번
    "남화토건",     # 31번
    "신원종합개발"  # 32번
]

# 22~32번 기업들의 주식코드 앵커링
STOCK_ANCHOR_22_32 = {
    "KCC건설": "021320",     # 22번 
    "HS화성": "002460",     # 23번
    "서한": "011370",       # 24번
    "HL D&I": "014790",     # 25번
    "한신공영": "004960",   # 26번
    "남광토건": "001260",   # 27번
    "삼부토건": "001470",   # 28번
    "우원개발": "046940",   # 29번
    "대원": "007680",       # 30번
    "남화토건": "091590",   # 31번
    "신원종합개발": "017000" # 32번
}

# 22~32번 기업들 설정 확인 및 corp_refs 생성
if 'CorpRef' in globals():
    print(" CorpRef 클래스가 정의되어 있습니다. 22~32번 기업 설정을 진행합니다.")
    
    # corp_refs 생성 (22~32번)
    corp_refs_22_32 = {name: CorpRef(None, name, None) for name in COMPANIES_22_32}
    
    # 22~32번 기업들 앵커링 수행
    print("=== 22~32번 기업들 앵커링 수행 ===")
    if '_resolve_by_stock' in globals():
        for nm, sc in STOCK_ANCHOR_22_32.items():
            try:
                cc, now_official = _resolve_by_stock(sc)
                corp_refs_22_32[nm] = CorpRef(cc, nm, sc)
                print(f"[anchor 22-32] {nm} -> corp_code={cc}, stock_code={sc}, now_official='{now_official}'")
            except Exception as e:
                print(f"[warn] anchor 22-32 {nm}: {e}")

        print(f"\n22~32번 대상 기업 수: {len(COMPANIES_22_32)}개")
    else:
        print(" _resolve_by_stock 함수가 정의되지 않았습니다.")
else:
    print(" CorpRef 클래스가 정의되지 않았습니다. 먼저 기본 셀을 실행해주세요.")

# =========================
# 22~32번 기업들 데이터 수집 시작
# =========================
print(f"\n=== 22~32번 기업 데이터 수집 시작 (2015년 Q4 ~ {END_YEAR}년) ===")

companies_22_32_dfs = []

for nm in tqdm(COMPANIES_22_32, desc="22-32번 Companies"):
    ref = corp_refs_22_32[nm]
    if ref.corp_code is None:
        print(f" {nm} - corp_code가 없음")
        continue
        
    print(f"\n {nm} 데이터 수집 중...")
    
    for yy in range(START_YEAR_LOAD, END_YEAR+1):
        try:
            df_y = yearly_quarter_table_with_lineage(nm, ref.corp_code, yy)
            companies_22_32_dfs.append(df_y)
        except Exception as e:
            print(f" {nm} {yy}년 실패: {e}")

# 22~32번 기업들 데이터 결합
if companies_22_32_dfs:
    result_df_22_32 = pd.concat(companies_22_32_dfs, ignore_index=True)
    result_df_22_32 = result_df_22_32[result_df_22_32["year"] >= 2015].copy()  # 2015년부터 사용
    
    # 컬럼 순서 맞추기
    result_df_22_32 = result_df_22_32[COL_ORDER].sort_values(["corp_name","year","quarter"]).reset_index(drop=True)
    
    # 2015년 1~3분기 제거 (2015년 4분기부터 시작)
    mask_2015_q1_q3_22_32 = (result_df_22_32["year"] == 2015) & (result_df_22_32["quarter"].isin(["Q1", "Q2", "Q3"]))
    
    # 현재년도 제한 적용
    if MAX_QUARTER_CURRENT_YEAR and END_YEAR == CURRENT_YEAR:
        if MAX_QUARTER_CURRENT_YEAR == "Q2":
            mask_current_year_limit_22_32 = (result_df_22_32["year"] == CURRENT_YEAR) & (result_df_22_32["quarter"].isin(["Q3", "Q4"]))
        elif MAX_QUARTER_CURRENT_YEAR == "Q3":
            mask_current_year_limit_22_32 = (result_df_22_32["year"] == CURRENT_YEAR) & (result_df_22_32["quarter"].isin(["Q4"]))
        else:
            mask_current_year_limit_22_32 = pd.Series([False] * len(result_df_22_32))
        
        to_drop_mask_22_32 = mask_2015_q1_q3_22_32 | mask_current_year_limit_22_32
    else:
        to_drop_mask_22_32 = mask_2015_q1_q3_22_32
    
    # 제거 대상 행 확인 및 제거
    to_drop_22_32 = result_df_22_32.loc[to_drop_mask_22_32, ["corp_name", "year", "quarter"]].copy()
    if len(to_drop_22_32) > 0:
        print(f" 22~32번 제거 대상: {len(to_drop_22_32)}개 레코드")
    
    result_df_22_32 = result_df_22_32.loc[~to_drop_mask_22_32].reset_index(drop=True)
    result_df_22_32 = result_df_22_32.sort_values(["corp_name", "year", "quarter"]).reset_index(drop=True)
    
    print(f"\n 22~32번 기업들 수집 완료!")
    print(f"   총 레코드: {len(result_df_22_32)}개")
    print(f"   기업 수: {result_df_22_32['corp_name'].nunique()}개")
    print(f"   연도 범위: {result_df_22_32['year'].min()}~{result_df_22_32['year'].max()}")
else:
    print(" 22~32번 기업들 데이터 수집 실패")

# 22~32번 기업들 데이터 저장
if 'result_df_22_32' in locals() and not result_df_22_32.empty:
    # 결측치 확인
    financial_cols = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']
    missing_22_32 = result_df_22_32[result_df_22_32[financial_cols].isna().any(axis=1)]
    
    if len(missing_22_32) > 0:
        print(f"\n 22~32번 결측치 현황: {len(missing_22_32)}개 레코드")
    
    # CSV 및 엑셀 파일 저장
    csv_path_22_32 = "./dart_out/건설업22~32번_2015~2025_연결_분기재무_정규화.csv"
    xlsx_path_22_32 = "./dart_out/건설업22~32번_2015~2025_연결_분기재무_정규화.xlsx"
    
    # CSV 저장
    result_df_22_32.to_csv(csv_path_22_32, index=False, encoding='utf-8-sig')
    print(f"\n 22~32번 CSV 저장: {csv_path_22_32}")
    
    # 엑셀 저장
    with pd.ExcelWriter(xlsx_path_22_32, engine='openpyxl') as w:
        result_df_22_32.to_excel(w, sheet_name='22-32번_건설업_분기재무', index=False)
    print(f" 22~32번 엑셀 저장: {xlsx_path_22_32}")
    
    print(f"\n 22~32번 기업들 최종 데이터:")
    print(f"   정리된 레코드: {len(result_df_22_32)}개")
    print(f"   기업별 분포:")
    for corp_name, count in result_df_22_32['corp_name'].value_counts().items():
        corp_num = COMPANIES_22_32.index(corp_name) + 22
        print(f"     {corp_num}. {corp_name}: {count}개")

print("\n 모든 기업 그룹 데이터 수집 및 저장 완료!")
print(" 저장된 파일:")
print("   - 건설업1~10번_2015~2025_연결_분기재무_정규화.csv/.xlsx")
print("   - 건설업11~21번_2015~2025_연결_분기재무_정규화.csv/.xlsx") 
print("   - 건설업22~32번_2015~2025_연결_분기재무_정규화.csv/.xlsx")

 CorpRef 클래스가 정의되어 있습니다. 22~32번 기업 설정을 진행합니다.
=== 22~32번 기업들 앵커링 수행 ===
[anchor 22-32] KCC건설 -> corp_code=00105466, stock_code=021320, now_official='KCC건설'
[anchor 22-32] HS화성 -> corp_code=00174527, stock_code=002460, now_official='HS화성'
[anchor 22-32] 서한 -> corp_code=00131504, stock_code=011370, now_official='서한'
[anchor 22-32] HL D&I -> corp_code=00161116, stock_code=014790, now_official='HL D&I'
[anchor 22-32] 한신공영 -> corp_code=00162063, stock_code=004960, now_official='한신공영'
[anchor 22-32] 남광토건 -> corp_code=00107066, stock_code=001260, now_official='남광토건'
[anchor 22-32] 삼부토건 -> corp_code=00125974, stock_code=001470, now_official='삼부토건'
[anchor 22-32] 우원개발 -> corp_code=00363246, stock_code=046940, now_official='우원개발'
[anchor 22-32] 대원 -> corp_code=00111838, stock_code=007680, now_official='대원'
[anchor 22-32] 남화토건 -> corp_code=00108001, stock_code=091590, now_official='남화토건'
[anchor 22-32] 신원종합개발 -> corp_code=00136925, stock_code=017000, now_official='신원종합개발'

22~32번 대상 기업 수: 11개

==

22-32번 Companies:   0%|          | 0/11 [00:00<?, ?it/s]


 KCC건설 데이터 수집 중...


22-32번 Companies:   9%|▉         | 1/11 [00:34<05:49, 34.97s/it]


 HS화성 데이터 수집 중...


...

22-32번 Companies:  91%|█████████ | 10/11 [04:48<00:27, 27.27s/it]


 신원종합개발 데이터 수집 중...


22-32번 Companies: 100%|██████████| 11/11 [05:23<00:00, 29.40s/it]

 22~32번 제거 대상: 55개 레코드

 22~32번 기업들 수집 완료!
   총 레코드: 429개
   기업 수: 11개
   연도 범위: 2015~2025

 22~32번 결측치 현황: 53개 레코드

 22~32번 CSV 저장: ./dart_out/건설업22~32번_2015~2025_연결_분기재무_정규화.csv
 22~32번 엑셀 저장: ./dart_out/건설업22~32번_2015~2025_연결_분기재무_정규화.xlsx

 22~32번 기업들 최종 데이터:
   정리된 레코드: 429개
   기업별 분포:
     25. HL D&I: 39개
     23. HS화성: 39개
     22. KCC건설: 39개
     27. 남광토건: 39개
     31. 남화토건: 39개
     30. 대원: 39개
     28. 삼부토건: 39개
     24. 서한: 39개
     32. 신원종합개발: 39개
     29. 우원개발: 39개
     26. 한신공영: 39개

 모든 기업 그룹 데이터 수집 및 저장 완료!
 저장된 파일:
   - 건설업1~10번_2015~2025_연결_분기재무_정규화.csv/.xlsx
   - 건설업11~21번_2015~2025_연결_분기재무_정규화.csv/.xlsx
   - 건설업22~32번_2015~2025_연결_분기재무_정규화.csv/.xlsx



C:\Users\baesh\AppData\Local\Temp\ipykernel_17552\2960666148.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df_22_32 = pd.concat(companies_22_32_dfs, ignore_index=True)


In [29]:
# =========================
# 전체 데이터 수집 결과 요약
# =========================

print("=" * 60)
print(" 건설업 기업 데이터 수집 결과 요약")
print("=" * 60)

# 각 그룹별 결과 확인
groups_summary = []

# 1~10번 그룹
if 'result_df_1_10' in globals() and not result_df_1_10.empty:
    groups_summary.append({
        'group': '1~10번',
        'companies': len(result_df_1_10['corp_name'].unique()),
        'records': len(result_df_1_10),
        'period': f"{result_df_1_10['year'].min()}.Q{result_df_1_10[result_df_1_10['year']==result_df_1_10['year'].min()]['quarter'].min()[1]} ~ {result_df_1_10['year'].max()}.Q{result_df_1_10[result_df_1_10['year']==result_df_1_10['year'].max()]['quarter'].max()[1]}"
    })

# 11~21번 그룹  
if 'result_df_11_21' in globals() and not result_df_11_21.empty:
    groups_summary.append({
        'group': '11~21번',
        'companies': len(result_df_11_21['corp_name'].unique()),
        'records': len(result_df_11_21),
        'period': f"{result_df_11_21['year'].min()}.Q{result_df_11_21[result_df_11_21['year']==result_df_11_21['year'].min()]['quarter'].min()[1]} ~ {result_df_11_21['year'].max()}.Q{result_df_11_21[result_df_11_21['year']==result_df_11_21['year'].max()]['quarter'].max()[1]}"
    })

# 22~32번 그룹
if 'result_df_22_32' in globals() and not result_df_22_32.empty:
    groups_summary.append({
        'group': '22~32번',
        'companies': len(result_df_22_32['corp_name'].unique()),
        'records': len(result_df_22_32),
        'period': f"{result_df_22_32['year'].min()}.Q{result_df_22_32[result_df_22_32['year']==result_df_22_32['year'].min()]['quarter'].min()[1]} ~ {result_df_22_32['year'].max()}.Q{result_df_22_32[result_df_22_32['year']==result_df_22_32['year'].max()]['quarter'].max()[1]}"
    })

# 요약 출력
total_companies = sum([g['companies'] for g in groups_summary])
total_records = sum([g['records'] for g in groups_summary])

print(f" 총 수집 기업 수: {total_companies}개")
print(f" 총 데이터 레코드: {total_records:,}개")
print(f" 수집 기간: 2015년 Q4 ~ {END_YEAR}년 {MAX_QUARTER_CURRENT_YEAR if MAX_QUARTER_CURRENT_YEAR else 'Q4'}")
print()

for group in groups_summary:
    print(f" {group['group']} 그룹: {group['companies']}개 기업, {group['records']:,}개 레코드")
    print(f"   기간: {group['period']}")

print()
print(" 생성된 파일 목록:")
print("   1. 건설업1~10번_2015~2025_연결_분기재무_정규화.csv/.xlsx")
print("   2. 건설업11~21번_2015~2025_연결_분기재무_정규화.csv/.xlsx")  
print("   3. 건설업22~32번_2015~2025_연결_분기재무_정규화.csv/.xlsx")
print()
print(" 모든 그룹별 데이터 수집 및 저장이 완료되었습니다!")

# 현재 날짜 기준 수집 범위 안내
current_date = datetime.now()
print(f"\n 참고사항:")
print(f"   - 현재 날짜: {current_date.strftime('%Y년 %m월 %d일')}")
print(f"   - 수집 기준: {current_date.month}월이므로 {MAX_QUARTER_CURRENT_YEAR if MAX_QUARTER_CURRENT_YEAR else '연말'}까지 수집")
if MAX_QUARTER_CURRENT_YEAR == "Q2":
    print(f"   - 2025년 3분기, 4분기 데이터는 아직 공시되지 않아 제외됨")

 건설업 기업 데이터 수집 결과 요약
 총 수집 기업 수: 32개
 총 데이터 레코드: 1,248개
 수집 기간: 2015년 Q4 ~ 2025년 Q2

 1~10번 그룹: 10개 기업, 390개 레코드
   기간: 2015.Q4 ~ 2025.Q2
 11~21번 그룹: 11개 기업, 429개 레코드
   기간: 2015.Q4 ~ 2025.Q2
 22~32번 그룹: 11개 기업, 429개 레코드
   기간: 2015.Q4 ~ 2025.Q2

 생성된 파일 목록:
   1. 건설업1~10번_2015~2025_연결_분기재무_정규화.csv/.xlsx
   2. 건설업11~21번_2015~2025_연결_분기재무_정규화.csv/.xlsx
   3. 건설업22~32번_2015~2025_연결_분기재무_정규화.csv/.xlsx

 모든 그룹별 데이터 수집 및 저장이 완료되었습니다!

 참고사항:
   - 현재 날짜: 2025년 09월 09일
   - 수집 기준: 9월이므로 Q2까지 수집
   - 2025년 3분기, 4분기 데이터는 아직 공시되지 않아 제외됨


In [38]:
# =========================
# 수기입력 데이터 병합 (3개 파일 통합)
# =========================

# 수기입력 데이터 로드 (3개 파일)
manual_files = [
    'dart_out/dart_결측치01.csv',
    'dart_out/dart_결측치02.csv', 
    'dart_out/dart_결측치03.csv'
]

manual_data_list = []
for file_path in manual_files:
    try:
        data = pd.read_csv(file_path)
        print(f"{file_path} 로드 완료: {data.shape}")
        manual_data_list.append(data)
    except FileNotFoundError:
        print(f"{file_path} 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"{file_path} 로드 실패: {e}")

# 수기입력 데이터 통합
if manual_data_list:
    manual_data = pd.concat(manual_data_list, ignore_index=True)
    print(f"통합 수기입력 데이터: {manual_data.shape}")
    
    # 중복 제거 (같은 corp_name, year, quarter 조합이 있다면 마지막 것 사용)
    manual_data = manual_data.drop_duplicates(subset=['corp_name', 'year', 'quarter'], keep='last')
    print(f"중복 제거 후: {manual_data.shape}")
else:
    print("수기입력 데이터를 로드할 수 없습니다.")
    manual_data = pd.DataFrame()

# 데이터가 있는 경우에만 병합 진행
if not manual_data.empty and 'result_df' in globals():
    print(f"병합 전 결측치 개수: {result_df.isnull().sum().sum()}")
    
    # 공통 컬럼으로 병합 (corp_name, year, quarter 기준)
    common_cols = ['corp_name', 'year', 'quarter']
    financial_cols = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']
    
    result_df_updated = result_df.merge(
        manual_data[common_cols + financial_cols], 
        on=common_cols, 
        how='left', 
        suffixes=('', '_manual')
    )
    
    # 결측치가 있는 곳에 수기입력 데이터로 보완
    for col in financial_cols:
        if f'{col}_manual' in result_df_updated.columns:
            result_df_updated[col] = result_df_updated[col].fillna(result_df_updated[f'{col}_manual'])
            result_df_updated.drop(f'{col}_manual', axis=1, inplace=True)
    
    result_df = result_df_updated
    print(f"병합 후 결측치 개수: {result_df.isnull().sum().sum()}")
    print(f"최종 데이터 shape: {result_df.shape}")
else:
    print("result_df가 없거나 수기입력 데이터가 비어있어 병합을 건너뜁니다.")

dart_out/dart_결측치01.csv 로드 완료: (390, 11)
dart_out/dart_결측치02.csv 로드 완료: (429, 11)
dart_out/dart_결측치03.csv 로드 완료: (429, 11)
통합 수기입력 데이터: (1248, 11)
중복 제거 후: (1248, 11)
병합 전 결측치 개수: 351
병합 후 결측치 개수: 351
최종 데이터 shape: (1248, 11)


In [37]:
pd.DataFrame.to_csv(result_df, 'dart_out/dart_merged_data.csv', index=False)

In [ ]:
# =========================
# DB 연결 및 저장 기능
# =========================

import sys
import os

# DB 폴더의 .env 파일을 명시적으로 로드
current_dir = os.getcwd()  # 현재 작업 디렉토리
project_root = os.path.dirname(current_dir)  # 프로젝트 루트
db_folder = os.path.join(project_root, 'DB')
env_file_path = os.path.join(db_folder, '.env')

print(f"현재 디렉토리: {current_dir}")
print(f"프로젝트 루트: {project_root}")
print(f"DB 폴더: {db_folder}")
print(f"환경변수 파일 경로: {env_file_path}")

if os.path.exists(env_file_path):
    from dotenv import load_dotenv
    load_dotenv(env_file_path)
    print(f".env 파일 로드 성공")
    print(f"DB_PASSWORD 설정됨: {'YES' if os.getenv('DB_PASSWORD') else 'NO'}")
else:
    print(f".env 파일을 찾을 수 없습니다: {env_file_path}")

# DB 연결을 위한 경로 추가 및 import
print(f"DB 폴더가 존재하는지 확인: {os.path.exists(db_folder)}")
if os.path.exists(db_folder):
    print(f"DB 폴더 내용: {os.listdir(db_folder)}")

# sys.path에 DB 폴더 추가
if db_folder not in sys.path:
    sys.path.insert(0, db_folder)
    print(f"sys.path에 DB 폴더 추가: {db_folder}")

try:
    # db_query.py 파일에서 DatabaseConnection 클래스 import
    db_query_file = os.path.join(db_folder, 'db_query.py')
    print(f"db_query.py 파일 존재: {os.path.exists(db_query_file)}")
    
    from db_query import DatabaseConnection
    print("DatabaseConnection 모듈 import 성공!")
    
except ImportError as e:
    print(f"db_query 모듈 import 실패: {e}")
    
    # 대안: 직접 경로에서 모듈 로드 시도
    try:
        import importlib.util
        spec = importlib.util.spec_from_file_location("db_query", db_query_file)
        db_query_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(db_query_module)
        DatabaseConnection = db_query_module.DatabaseConnection
        print("직접 로드로 DatabaseConnection 모듈 import 성공!")
    except Exception as e2:
        print(f"직접 로드도 실패: {e2}")
        DatabaseConnection = None

except Exception as e:
    print(f"예상치 못한 오류: {e}")
    DatabaseConnection = None

def save_to_database_incremental(df):
    """
    DART 재무데이터를 MySQL 데이터베이스에 증분 저장 (기존 데이터 유지, 없는 것만 추가)
    """
    if DatabaseConnection is None:
        print("데이터베이스 모듈을 사용할 수 없습니다. CSV 파일로만 저장됩니다.")
        return False
        
    try:
        db = DatabaseConnection()
        if not db.connect():
            print("데이터베이스 연결 실패 - CSV 파일로만 저장됩니다.")
            return False
        
        cursor = db.connection.cursor()
        
        # 기존 데이터 확인
        cursor.execute("SELECT COUNT(*) FROM dart_data")
        existing_count = cursor.fetchone()[0]
        print(f"기존 DB 레코드 수: {existing_count}개")
        
        # 기존 데이터의 키 조합 가져오기 (corp_code, year, quarter)
        cursor.execute("""
            SELECT DISTINCT corp_code, year, quarter 
            FROM dart_data
        """)
        existing_keys = set(cursor.fetchall())
        print(f"기존 데이터 키 조합 수: {len(existing_keys)}개")
        
        # 데이터 변환 및 정제
        df_clean = df.copy()
        print(f"=== 신규 데이터 확인 ({len(df_clean)}개 레코드) ===")
        
        # report_date를 DATE 형식으로 변환
        df_clean['report_date'] = pd.to_datetime(df_clean['report_date']).dt.date
        
        # 재무 데이터를 적절한 숫자 형식으로 변환
        financial_columns = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']
        for col in financial_columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        
        # 신규 데이터 필터링 (기존에 없는 것만)
        new_records = []
        duplicate_count = 0
        
        for _, row in df_clean.iterrows():
            key = (row['corp_code'], int(row['year']), row['quarter'])
            if key not in existing_keys:
                new_records.append(row)
            else:
                duplicate_count += 1
        
        print(f"중복 데이터 (건너뜀): {duplicate_count}개")
        print(f"신규 추가할 데이터: {len(new_records)}개")
        
        if not new_records:
            print("추가할 신규 데이터가 없습니다.")
            cursor.close()
            db.disconnect()
            return True
        
        # 신규 데이터를 DataFrame으로 변환
        df_new = pd.DataFrame(new_records)
        
        # MySQL 삽입 쿼리
        insert_query = """
        INSERT INTO dart_data (
            corp_name, corp_code, year, quarter, report_date,
            total_assets, total_liabilities, total_equity,
            revenue, operating_profit, quarterly_profit
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # 배치 저장
        batch_size = 50
        total_batches = (len(df_new) + batch_size - 1) // batch_size
        
        print(f"=== 신규 데이터 저장 시작 ({len(df_new)}개 레코드) ===")
        
        for i in range(0, len(df_new), batch_size):
            batch_df = df_new.iloc[i:i + batch_size]
            batch_num = (i // batch_size) + 1
            
            print(f"배치 {batch_num}/{total_batches} 저장 중... ({len(batch_df)}개)")
            
            # 배치 데이터 준비 - NaN 값을 안전하게 처리
            batch_data = []
            for _, row in batch_df.iterrows():
                def safe_value(val):
                    """NaN, inf, 문자열 'nan' 등을 None으로 안전하게 변환"""
                    if pd.isna(val) or val is None:
                        return None
                    if isinstance(val, str) and val.lower() in ['nan', 'inf', '-inf', 'null']:
                        return None
                    try:
                        if pd.isna(pd.Series([val]).iloc[0]):
                            return None
                        return float(val) if val != '' else None
                    except:
                        return None
                
                batch_data.append((
                    row['corp_name'],
                    row['corp_code'],
                    int(row['year']),
                    row['quarter'],
                    row['report_date'],
                    safe_value(row['자산총계']),
                    safe_value(row['부채총계']),
                    safe_value(row['자본총계']),
                    safe_value(row['매출액']),
                    safe_value(row['영업이익']),
                    safe_value(row['분기순이익'])
                ))
            
            cursor.executemany(insert_query, batch_data)
            db.connection.commit()
            print(f"배치 {batch_num} 저장 완료")
            
            if i + batch_size < len(df_new):
                time.sleep(0.3)
        
        print(f"신규 {len(df_new)}개 레코드 추가 완료!")
        
        # 최종 결과 확인
        cursor.execute("SELECT COUNT(*) FROM dart_data")
        final_count = cursor.fetchone()[0]
        added_count = final_count - existing_count
        print(f"최종 DB 레코드 수: {final_count}개 (추가됨: {added_count}개)")
        
        cursor.close()
        db.disconnect()
        return True
        
    except Exception as e:
        print(f"데이터베이스 저장 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        if 'cursor' in locals():
            cursor.close()
        if 'db' in locals():
            db.disconnect()
        return False

def check_database_results():
    """데이터베이스에 저장된 DART 데이터 확인"""
    if DatabaseConnection is None:
        print("데이터베이스 모듈을 사용할 수 없습니다.")
        return
    
    try:
        db = DatabaseConnection()
        if not db.connect():
            print("데이터베이스 연결 실패")
            return
        
        cursor = db.connection.cursor()
        
        # 총 레코드 수 확인
        cursor.execute("SELECT COUNT(*) FROM dart_data")
        total_count = cursor.fetchone()[0]
        print(f"총 저장된 레코드 수: {total_count}")
        
        # 기업별 레코드 수 확인
        cursor.execute("""
            SELECT corp_name, COUNT(*) as record_count 
            FROM dart_data 
            GROUP BY corp_name 
            ORDER BY record_count DESC
        """)
        corp_counts = cursor.fetchall()
        print(f"\n기업별 레코드 수:")
        for corp_name, count in corp_counts[:10]:
            print(f"  {corp_name}: {count}개")
        
        # 연도별 레코드 수 확인
        cursor.execute("""
            SELECT year, COUNT(*) as record_count 
            FROM dart_data 
            GROUP BY year 
            ORDER BY year DESC
        """)
        year_counts = cursor.fetchall()
        print(f"\n연도별 레코드 수:")
        for year, count in year_counts:
            print(f"  {year}년: {count}개")
        
        # 최근 데이터 샘플 확인
        cursor.execute("""
            SELECT corp_name, year, quarter, revenue, operating_profit 
            FROM dart_data 
            ORDER BY year DESC, quarter DESC 
            LIMIT 5
        """)
        recent_data = cursor.fetchall()
        print(f"\n최근 데이터 샘플:")
        for corp_name, year, quarter, revenue, operating_profit in recent_data:
            revenue_str = f"{revenue:,.0f}" if revenue else "N/A"
            profit_str = f"{operating_profit:,.0f}" if operating_profit else "N/A"
            print(f"  {corp_name} {year}년 {quarter}: 매출액 {revenue_str}, 영업이익 {profit_str}")
        
        cursor.close()
        db.disconnect()
        
    except Exception as e:
        print(f"데이터베이스 확인 중 오류: {e}")

def load_from_csv_and_save_to_db():
    """CSV 파일에서 데이터를 읽어와 데이터베이스에 증분 저장"""
    csv_files = [
        "./dart_out/dart_merged_data.csv"
    ]
    
    df_loaded = None
    for csv_file in csv_files:
        if os.path.exists(csv_file):
            try:
                df_loaded = pd.read_csv(csv_file, encoding='utf-8-sig')
                print(f"CSV 파일 로드 성공: {csv_file}")
                break
            except Exception as e:
                print(f"CSV 파일 로드 실패: {csv_file} - {e}")
    
    if df_loaded is None:
        print("사용 가능한 CSV 파일을 찾을 수 없습니다.")
        return False
    
    print(f"CSV 데이터 정보: {len(df_loaded)}개 레코드")
    
    # 데이터베이스에 증분 저장
    db_success = save_to_database_incremental(df_loaded)
    
    if db_success:
        print(f"\n=== 저장 완료! 결과 확인 ===")
        check_database_results()
        return True
    else:
        print(f"\n데이터베이스 저장 실패")
        return False

print("DB 연결 및 저장 함수 정의 완료")

현재 디렉토리: c:\Users\baesh\Desktop\kor-ie-proj\dart
프로젝트 루트: c:\Users\baesh\Desktop\kor-ie-proj
DB 폴더: c:\Users\baesh\Desktop\kor-ie-proj\DB
환경변수 파일 경로: c:\Users\baesh\Desktop\kor-ie-proj\DB\.env
.env 파일 로드 성공
DB_PASSWORD 설정됨: YES
DB 폴더가 존재하는지 확인: True
DB 폴더 내용: ['.env', 'db_query.py', 'ddl.sql', 'README.md', '__pycache__']
db_query.py 파일 존재: True
DatabaseConnection 모듈 import 성공!
DB 연결 및 저장 함수 정의 완료


In [44]:
# =========================
# 데이터베이스 저장 실행
# =========================

# result_df가 이미 존재하는지 확인
if 'result_df' in globals() and not result_df.empty:
    print(f"수집된 데이터 확인:")
    print(f"  - 레코드 수: {len(result_df)}")
    print(f"  - 기업 수: {result_df['corp_name'].nunique()}")
    print(f"  - 연도 범위: {result_df['year'].min()} ~ {result_df['year'].max()}")
    
    # 데이터베이스에 증분 저장
    db_success = save_to_database_incremental(result_df)
    
    if db_success:
        print(f"\n=== 저장 완료! 결과 확인 ===")
        check_database_results()
    else:
        print(f"\n데이터베이스 저장 실패 - CSV에서 로드 시도")
        load_from_csv_and_save_to_db()
        
else:
    print("result_df가 존재하지 않습니다. CSV에서 로드하여 저장을 시도합니다.")
    load_from_csv_and_save_to_db()

수집된 데이터 확인:
  - 레코드 수: 1248
  - 기업 수: 32
  - 연도 범위: 2015 ~ 2025
MySQL 데이터베이스 연결 성공
기존 DB 레코드 수: 390개
기존 데이터 키 조합 수: 390개
=== 신규 데이터 확인 (1248개 레코드) ===
중복 데이터 (건너뜀): 390개
신규 추가할 데이터: 858개
=== 신규 데이터 저장 시작 (858개 레코드) ===
배치 1/18 저장 중... (50개)
배치 1 저장 완료
배치 2/18 저장 중... (50개)
배치 2 저장 완료
배치 2/18 저장 중... (50개)
배치 2 저장 완료
배치 3/18 저장 중... (50개)
배치 3 저장 완료
배치 3/18 저장 중... (50개)
배치 3 저장 완료
배치 4/18 저장 중... (50개)
배치 4 저장 완료
배치 4/18 저장 중... (50개)
배치 4 저장 완료
배치 5/18 저장 중... (50개)
배치 5 저장 완료
배치 5/18 저장 중... (50개)
배치 5 저장 완료
배치 6/18 저장 중... (50개)
배치 6 저장 완료
배치 6/18 저장 중... (50개)
배치 6 저장 완료
배치 7/18 저장 중... (50개)
배치 7 저장 완료
배치 7/18 저장 중... (50개)
배치 7 저장 완료
배치 8/18 저장 중... (50개)
배치 8 저장 완료
배치 8/18 저장 중... (50개)
배치 8 저장 완료
배치 9/18 저장 중... (50개)
배치 9 저장 완료
배치 9/18 저장 중... (50개)
배치 9 저장 완료
배치 10/18 저장 중... (50개)
배치 10 저장 완료
배치 10/18 저장 중... (50개)
배치 10 저장 완료
배치 11/18 저장 중... (50개)
배치 11 저장 완료
배치 11/18 저장 중... (50개)
배치 11 저장 완료
배치 12/18 저장 중... (50개)
배치 12 저장 완료
배치 12/18 저장 중... (50개)
배치 12 저장 완료
배치 13/18 저